<a href="https://colab.research.google.com/github/newmantic/CMA-ES/blob/main/CMA_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class CMAES:
    def __init__(self, func, n_dim, population_size=10, sigma=0.5, max_iter=1000):
        self.func = func
        self.n_dim = n_dim
        self.population_size = population_size
        self.sigma = sigma
        self.max_iter = max_iter

        # Initialize the mean vector and covariance matrix
        self.mean = np.random.rand(n_dim)
        self.cov = np.eye(n_dim)

        # Initialize strategy parameters
        self.weights = np.log(self.population_size / 2 + 1) - np.log(np.arange(1, self.population_size + 1))
        self.weights /= np.sum(self.weights)
        self.mueff = 1 / np.sum(self.weights ** 2)
        self.cc = (4 + self.mueff / n_dim) / (n_dim + 4 + 2 * self.mueff / n_dim)
        self.cs = (self.mueff + 2) / (n_dim + self.mueff + 5)
        self.c1 = 2 / ((n_dim + 1.3) ** 2 + self.mueff)
        self.cmu = min(1 - self.c1, 2 * (self.mueff - 2 + 1 / self.mueff) / ((n_dim + 2) ** 2 + self.mueff))
        self.damps = 1 + 2 * max(0, np.sqrt((self.mueff - 1) / (n_dim + 1)) - 1) + self.cs

        # Initialize paths
        self.pc = np.zeros(n_dim)
        self.ps = np.zeros(n_dim)
        self.B = np.eye(n_dim)
        self.D = np.ones(n_dim)
        self.C = np.eye(n_dim)
        self.invsqrtC = np.eye(n_dim)
        self.eigen_eval = 0
        self.chiN = np.sqrt(n_dim) * (1 - 1 / (4 * n_dim) + 1 / (21 * n_dim ** 2))

    def optimize(self):
        for generation in range(self.max_iter):
            # Generate a population
            population = np.array([self.mean + self.sigma * self.B @ (self.D * np.random.randn(self.n_dim))
                                   for _ in range(self.population_size)])
            fitness = np.array([self.func(ind) for ind in population])
            indices = np.argsort(fitness)
            population = population[indices]
            fitness = fitness[indices]

            # Update mean
            self.mean = np.dot(self.weights, population[:self.population_size])

            # Update evolution paths
            y = (self.mean - self.mean) / self.sigma
            self.ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * self.invsqrtC @ y
            hsig = np.linalg.norm(self.ps) / np.sqrt(1 - (1 - self.cs) ** (2 * (generation + 1))) / self.chiN < 1.4 + 2 / (self.n_dim + 1)
            self.pc = (1 - self.cc) * self.pc + hsig * np.sqrt(self.cc * (2 - self.cc) * self.mueff) * y

            # Update covariance matrix
            artmp = (1 / self.sigma) * (population[:self.population_size] - self.mean)
            self.C = (1 - self.c1 - self.cmu) * self.C + self.c1 * (np.outer(self.pc, self.pc) + (1 - hsig) * self.cc * (2 - self.cc) * self.C) + self.cmu * artmp.T @ np.diag(self.weights) @ artmp

            # Update step-size
            self.sigma *= np.exp((self.cs / self.damps) * (np.linalg.norm(self.ps) / self.chiN - 1))

            # Decompose the covariance matrix if needed
            if generation - self.eigen_eval > self.population_size / (self.c1 + self.cmu) / self.n_dim / 10:
                self.eigen_eval = generation
                self.C = np.triu(self.C) + np.triu(self.C, 1).T
                self.D, self.B = np.linalg.eigh(self.C)
                self.D = np.sqrt(self.D)
                self.invsqrtC = self.B @ np.diag(1 / self.D) @ self.B.T

            # Check for convergence
            if np.linalg.norm(fitness[0] - fitness[-1]) < 1e-8:
                break

        return self.mean

In [2]:
def sphere_function(x):
    return np.sum(x ** 2)

# Set the dimension of the problem
n_dim = 5

# Instantiate the CMA-ES optimizer
cmaes = CMAES(func=sphere_function, n_dim=n_dim, population_size=10, sigma=0.5, max_iter=1000)

# Perform optimization
optimal_solution = cmaes.optimize()
print("Optimal Solution:", optimal_solution)
print("Function Value at Optimal Solution:", sphere_function(optimal_solution))

Optimal Solution: [-1.01855044e-05  8.91807957e-07  9.05891034e-06  2.33830982e-05
  1.90115564e-05]
Function Value at Optimal Solution: 1.0948122382610533e-09
